In [ ]:
"""
MERSCOPE dir
------------
├── cell_boundaries/
│   ├── feature_data_#.hdf5
│   ├── ...
│   └── feature_data_n.hdf5
├── cell_by_gene.csv
├── cell_metadata.csv
├── detected_transcripts.csv
└── images/
    ├── micron_to_mosaic_pixel_transform.csv
    ├── mosaic_[type]_z#.tif
    ├── ...
    └── mosaic_[type]_zn.tif
"""

In [ ]:
merscope_dir = "/path/to/directory/"

### Generate h5ad file

In [ ]:
import os
import numpy as np
import pandas as pd
import anndata as ad

In [ ]:
out_dir = "./"
out_h5ad_filename = "merscope"

In [ ]:
m = pd.read_csv(os.path.join(merscope_dir, "cell_by_gene.csv"), index_col=0)
m = m.loc[:,~m.columns.str.startswith('Blank-')]
m.index = m.index.astype(str)

In [ ]:
cells = pd.read_csv(os.path.join(merscope_dir, "cell_metadata.csv"), index_col=0)
cells.index = cells.index.astype(str)

In [ ]:
adata = ad.AnnData(
    m.to_numpy(),
    dtype=np.float32,
    obs=cells.loc[m.index],
    var=m.columns.to_frame(name="gene")
    )

adata.obs["total_counts"] = adata.X.sum(axis=1)
adata.obs["n_genes_by_counts"] = (adata.X > 0).sum(axis=1)

adata.var["total_counts"] = adata.X.sum(axis=0)
adata.var["n_cells_by_counts"] = (adata.X > 0).sum(axis=0)

In [ ]:
tm = pd.read_csv(
    os.path.join(merscope_dir, "images", "micron_to_mosaic_pixel_transform.csv"),
    sep=" ",
    header=None,
    dtype=float
    ).values

sp_coords = adata.obs[["center_x", "center_y"]].values
sp_coords[:,0] = sp_coords[:,0] * tm[0, 0] + tm[0, 2]
sp_coords[:,1] = sp_coords[:,1] * tm[1, 1] + tm[1, 2]

adata.obsm["spatial"] = sp_coords

In [ ]:
adata.write_h5ad(os.path.join(out_dir, out_h5ad_filename + ".h5ad"))

### Concatenate raw tif images

In [ ]:
import os
import pyvips

In [ ]:
out_dir = "./"

# z indices to process
process_z = {0} # or None for all

In [ ]:
imgs = [ x for x in os.listdir(os.path.join(merscope_dir, "images")) if x.endswith(".tif") ]
imgs.sort()

z_imgs = set([ os.path.splitext(x)[0].split("_")[2] for x in imgs ])
if process_z and len(process_z):
    z_imgs = z_imgs.intersection(set([ "z{}".format(x) for x in process_z]))

In [ ]:
for z in z_imgs:
    t_imgs = [ x for x in imgs if x.endswith("{}.tif".format(z)) ]
    channels = [ x.split("_")[1] for x in t_imgs ]

    v_imgs = [
        pyvips.Image.new_from_file(os.path.join(merscope_dir, "images", x), access="sequential")
        for x in t_imgs
    ]
    z_img = pyvips.Image.arrayjoin(v_imgs, across=1)
    z_img = z_img.copy()
    z_img.set_type(pyvips.GValue.gint_type, "page-height", v_imgs[0].height)

    xml_channels = "".join([ f"""<Channel ID="Channel:0:{x}" SamplesPerPixel="1" Name="{c}"><LightPath/></Channel>""" for x,c in enumerate(channels) ])

    z_img.set_type(pyvips.GValue.gstr_type, "image-description", 
        " ".join(f"""<?xml version="1.0" encoding="UTF-8"?>
        <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
            <Image ID="Image:0">
                <Pixels DimensionOrder="XYCZT"
                        ID="Pixels:0"
                        SizeC="{len(v_imgs)}"
                        SizeT="1"
                        SizeX="{v_imgs[0].width}"
                        SizeY="{v_imgs[0].height}"
                        SizeZ="1"
                        Type="uint16">
                        {xml_channels}
                </Pixels>
            </Image>
        </OME>""".split()))

    print("Writing tif image {} ...".format(z))
    z_img.tiffsave(os.path.join(out_dir, "mosaic_{}.tif".format(z)), bigtiff=True)

### Generate label image

In [ ]:
import os
import h5py
import tifffile
import numpy as np
import pandas as pd
from skimage.draw import polygon

In [ ]:
out_dir = "./"

# z indices to process
process_z = {0}

In [ ]:
raw_img = [ x for x in os.listdir(os.path.join(merscope_dir, "images")) if x.endswith(".tif")][0]
tif = tifffile.TiffFile(os.path.join(merscope_dir, "images", raw_img))
height, width = tif.pages[0].shape

In [ ]:
tm = pd.read_csv(
    os.path.join(merscope_dir, "images", "micron_to_mosaic_pixel_transform.csv"),
    sep=" ",
    header=None,
    dtype=float
    ).values

In [ ]:
fovs = [ x for x in os.listdir(os.path.join(merscope_dir, "cell_boundaries")) if x.endswith(".hdf5") ]

for z, z_index in [ (x, "zIndex_{}".format(x)) for x in process_z ]:

    label = np.zeros((height, width), dtype=np.uint32)

    for fov in fovs:
        with h5py.File(os.path.join(merscope_dir, "cell_boundaries", fov)) as f:
            for cell_id in f["featuredata"].keys():
                pol = f["featuredata"][cell_id][z_index]["p_0"]["coordinates"][0]

                pol[:,0] = pol[:,0] * tm[0,0] + tm[0,2]
                pol[:,1] = pol[:,1] * tm[1,1] + tm[1,2]

                rr, cc = polygon(pol[:,1], pol[:,0])
                label[rr-1, cc-1] = int(cell_id)
                
    print("Writing label tif image {} ...".format(z))
    tifffile.imwrite(os.path.join(out_dir, "label_mosaic_z{}.tif".format(z)), label)